# FedPSO (clientes com dados completos)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms
import copy
import random
import csv
import torchvision.models as models

In [2]:
#Hiperparâmetros
BATCH_SIZE = 64
epocas_cliente = 5
num_clientes = 10
num_rodadas = 30
inertia, c1, c2 = 0.9, 0.8, 0.9

In [3]:
# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Função para inicializar o modelo de acordo com a arquitetura
def init_model(architecture="squeezenet"):
    if architecture == "squeezenet":
        model = models.squeezenet1_1(weights=None)
        model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
    elif architecture == "mobilenet_v2":
        model = models.mobilenet_v2(weights=None)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
    elif architecture == "resnet18":
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif architecture == "alexnet":
        model = models.alexnet(weights=None)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    else:
        raise ValueError("Unsupported architecture. Choose from 'squeezenet', 'mobilenet_v2', 'resnet18', 'alexnet'.")
    return model.to(device)

# Função para criar um subconjunto aleatório de um dataset
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)

# Função para salvar os resultados em um arquivo CSV
def write_csv(algorithm_name, architecture, results):
    file_name = f'{algorithm_name}_{architecture}_do zero_CIFAR10_output.csv'
    with open(file_name, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Epoch", "Test Loss", "Test Accuracy"])
        for i, result in enumerate(results):
            writer.writerow([i + 1] + result)

class Particle:
    def __init__(self, particle_id, model, data, architecture="squeezenet"):
        self.particle_id = particle_id
        self.model = model.to(device)
        self.data = data  # `self.data` agora armazena o dataset, não o DataLoader
        self.local_best_model = copy.deepcopy(model.state_dict())
        self.local_best_score = float('inf')
        self.velocity = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001, weight_decay=1e-5)

    def train_particle(self, criterion):
        self.model.train()
        train_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=True)
        for _ in range(epocas_cliente):
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

        val_loss = self.evaluate_loss(criterion)
        if val_loss < self.local_best_score:
            self.local_best_score = val_loss
            self.local_best_model = copy.deepcopy(self.model.state_dict())
        return self.particle_id, val_loss

    def evaluate_loss(self, criterion):
        self.model.eval()
        total_loss = 0
        val_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=False)
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(val_loader)

    def pso_update(self, global_best_model, inertia, c1, c2):
        for name, param in self.model.named_parameters():
            if param.grad is None:
                continue
            local_rand = random.random()
            global_rand = random.random()
            self.velocity[name] = (
                inertia * self.velocity[name]
                + c1 * local_rand * (self.local_best_model[name].to(device) - param.data)
                + c2 * global_rand * (global_best_model[name].to(device) - param.data)
            )
            param.data += self.velocity[name]

def train_federated(architecture="squeezenet"):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloaders = [create_subset(trainset, 50000) for _ in range(num_clientes)]

    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    server_model = init_model(architecture)
    particles = [Particle(i, copy.deepcopy(server_model), trainloaders[i], architecture) for i in range(num_clientes)]
    criterion = nn.CrossEntropyLoss()

    global_best_score = float('inf')
    global_best_model = copy.deepcopy(server_model.state_dict())
    server_evaluate_acc = []

    for epoch in range(num_rodadas):
        server_result = []
        for particle in particles:
            if epoch > 0:
                particle.pso_update(global_best_model, inertia, c1, c2)
            pid, val_loss = particle.train_particle(criterion)
            server_result.append((pid, val_loss))

        best_particle = min(server_result, key=lambda x: x[1])
        global_best_score = best_particle[1]
        global_best_model = copy.deepcopy(particles[best_particle[0]].local_best_model)

        print(f'Epoch {epoch + 1}/{num_rodadas}, Best Particle Loss: {global_best_score:.4f}')

        server_model.load_state_dict(global_best_model)
        server_model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = server_model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
        test_loss = total_loss / len(testloader)
        test_accuracy = correct / len(testset)
        server_evaluate_acc.append([test_loss, test_accuracy])
        print(f'Epoch {epoch + 1}/{num_rodadas}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    write_csv("FedPSO", architecture, server_evaluate_acc)

# Exemplo de execução
#train_federated("squeezenet")

## SqueezeNet

In [5]:
train_federated("squeezenet")  

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 1.1018
Epoch 1/30, Test Loss: 1.1288, Test Accuracy: 0.5889
Epoch 2/30, Best Particle Loss: 0.8845
Epoch 2/30, Test Loss: 0.9297, Test Accuracy: 0.6725
Epoch 3/30, Best Particle Loss: 0.7477
Epoch 3/30, Test Loss: 0.8199, Test Accuracy: 0.7161
Epoch 4/30, Best Particle Loss: 0.6404
Epoch 4/30, Test Loss: 0.7425, Test Accuracy: 0.7423
Epoch 5/30, Best Particle Loss: 0.5419
Epoch 5/30, Test Loss: 0.6888, Test Accuracy: 0.7627
Epoch 6/30, Best Particle Loss: 0.4803
Epoch 6/30, Test Loss: 0.6554, Test Accuracy: 0.7726
Epoch 7/30, Best Particle Loss: 0.3993
Epoch 7/30, Test Loss: 0.6234, Test Accuracy: 0.7932
Epoch 8/30, Best Particle Loss: 0.3618
Epoch 8/30, Test Loss: 0.6722, Test Accuracy: 0.7838
Epoch 9/30, Best Particle Loss: 0.2919
Epoch 9/30, Test Loss: 0.6702, Test Accuracy: 0.7927
Epoch 10/30, Best Particle Loss: 0.2432
Epoch 10/30, Test Loss: 0.7156, Test Accuracy: 0.7941
Ep

## MobileNet_v2

In [5]:
train_federated("mobilenet_v2") 

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 0.6889
Epoch 1/30, Test Loss: 0.8395, Test Accuracy: 0.7000
Epoch 2/30, Best Particle Loss: 0.2816
Epoch 2/30, Test Loss: 0.7360, Test Accuracy: 0.7580
Epoch 3/30, Best Particle Loss: 0.0932
Epoch 3/30, Test Loss: 0.7963, Test Accuracy: 0.7699
Epoch 4/30, Best Particle Loss: 0.0483
Epoch 4/30, Test Loss: 0.7968, Test Accuracy: 0.7894
Epoch 5/30, Best Particle Loss: 0.0296
Epoch 5/30, Test Loss: 0.9212, Test Accuracy: 0.7831
Epoch 6/30, Best Particle Loss: 0.0239
Epoch 6/30, Test Loss: 0.9221, Test Accuracy: 0.8002
Epoch 7/30, Best Particle Loss: 0.0164
Epoch 7/30, Test Loss: 0.9488, Test Accuracy: 0.7967
Epoch 8/30, Best Particle Loss: 0.0157
Epoch 8/30, Test Loss: 0.9570, Test Accuracy: 0.8011
Epoch 9/30, Best Particle Loss: 0.0114
Epoch 9/30, Test Loss: 0.9503, Test Accuracy: 0.8044
Epoch 10/30, Best Particle Loss: 0.0114
Epoch 10/30, Test Loss: 0.9646, Test Accuracy: 0.8052
Ep

## Resnet18

In [ ]:
train_federated("resnet18") 

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 0.2197
Epoch 1/30, Test Loss: 0.6225, Test Accuracy: 0.7922
Epoch 2/30, Best Particle Loss: 0.0868
Epoch 2/30, Test Loss: 0.7900, Test Accuracy: 0.7902
Epoch 3/30, Best Particle Loss: 0.0485
Epoch 3/30, Test Loss: 0.7818, Test Accuracy: 0.7979
Epoch 4/30, Best Particle Loss: 0.0282
Epoch 4/30, Test Loss: 0.7982, Test Accuracy: 0.8169
Epoch 5/30, Best Particle Loss: 0.0194
Epoch 5/30, Test Loss: 0.8011, Test Accuracy: 0.8156
Epoch 6/30, Best Particle Loss: 0.0116
Epoch 6/30, Test Loss: 0.7732, Test Accuracy: 0.8224
Epoch 7/30, Best Particle Loss: 0.0181
Epoch 7/30, Test Loss: 0.8021, Test Accuracy: 0.8166
Epoch 8/30, Best Particle Loss: 0.0109
Epoch 8/30, Test Loss: 0.7741, Test Accuracy: 0.8228
Epoch 9/30, Best Particle Loss: 0.0082
Epoch 9/30, Test Loss: 0.7670, Test Accuracy: 0.8241
Epoch 10/30, Best Particle Loss: 0.0137
Epoch 10/30, Test Loss: 0.8177, Test Accuracy: 0.8191
Ep

## AlexNet

In [ ]:
train_federated("alexnet") 

Files already downloaded and verified
Files already downloaded and verified
